Сиквенс белка - MLARALLLCAVLALSHTANPCCSHPCQNRGVCMSVGFDQYKCDCTRTGFYGENCSTPEFLTRIKLFLKPTPNTVHYILTHFKGFWNVVNNIPFLRNAIMSYVLTSRSHLIDSPPTYNADYGYKSWEAFSNLSYYTRALPPVPDDCPTPLGVKGKKQLPDSNEIVEKLLLRRKFIPDPQGSNMMFAFFAQHFTHQFFKTDHKRGPAFTNGLGHGVDLNHIYGETLARQRKLRLFKDGKMKYQIIDGEMYPPTVKDTQAEMIYPPQVPEHLRFAVGQEVFGLVPGLMMYATIWLREHNRVCDVLKQEHPEWGDEQLFQTSRLILIGETIKIVIEDYVQHLSGYHFKLKFDPELLFNKQFQYQNRIAAEFNTLYHWHPLLPDTFQIHDQKYNYQQFIYNNSILLEHGITQFVESFTRQIAGRVAGGRNVPPAVQKVSQASIDQSRQMKYQSFNEYRKRFMLKPYESFEELTGEKEMSAELEALYGDIDAVELYPALLVEKPRPDAIFGETMVEVGAPFSLKGLMGNVICSPAYWKPSTFGGEVGFQIINTASIQSLICNNVKGCPFTSFSVPDPELIKTVTINASSSRSGLDDINPTVLLKERSTEL

In [20]:
import numpy as np
import pandas as pd
from torch import nn
import torch

import mordred as md
from mordred import descriptors
import rdkit

In [30]:
mols = pd.read_csv('ligand_output/hash_ligand_mapping.csv', header = None)

mols.columns = ['file_name', 'smiles']

mols.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   file_name  100 non-null    object
 1   smiles     100 non-null    object
dtypes: object(2)
memory usage: 1.7+ KB


In [31]:
molecules = []
for i in range(len(mols)):
    molecules.append(rdkit.Chem.MolFromSmiles(mols['smiles'][i]))

calc = md.Calculator(descriptors, ignore_3D=True)
        
mordred_desc = calc.pandas(molecules)

file = pd.read_csv('Prep_data.csv')
mordred_desc = mordred_desc[file.iloc[:, :132].columns]

with open('Generated.csv', 'w', newline='', encoding='utf-8'):
    mordred_desc.to_csv('Generated.csv', index = False)

mordred_desc.info()

100%|██████████| 100/100 [00:03<00:00, 25.80it/s]


<class 'mordred._base.pandas_module.MordredDataFrame'>
RangeIndex: 100 entries, 0 to 99
Columns: 132 entries, nAcid to AXp-1dv
dtypes: float64(108), int64(24)
memory usage: 103.3 KB


In [14]:
# Будем использовать графовую нейросеть для предсказания

class DeepMLP(nn.Module):
    def __init__(self, in_size, hid_size, out_size, dropout = 0.5):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(in_size, hid_size),
                                 nn.Tanh(),
                                 nn.Dropout(dropout),
                                 nn.LayerNorm(hid_size),

                                 nn.Linear(hid_size, hid_size // 2),
                                 nn.Tanh(),
                                 nn.Dropout(dropout),
                                 nn.LayerNorm(hid_size // 2),

                                 nn.Linear(hid_size // 2, hid_size // 4),
                                 nn.Tanh(),
                                 nn.Dropout(dropout),
                                 nn.LayerNorm(hid_size // 4),
                                 
                                 nn.Linear(hid_size // 4, hid_size // 8),
                                 nn.Tanh(),
                                 nn.Dropout(dropout),
                                 nn.LayerNorm(hid_size // 8),
                                 
                                 nn.Linear(hid_size // 8, hid_size // 16),
                                 nn.Tanh(),
                                 nn.Dropout(dropout),
                                 nn.LayerNorm(hid_size // 16),

                                 nn.Linear(hid_size // 16, out_size))

    def forward(self, x):
        return self.net(x)
    
model = DeepMLP(in_size = 132, hid_size = 132, out_size = 1)

model.load_state_dict(torch.load('best_regressor.pth'))

model.eval()

DeepMLP(
  (net): Sequential(
    (0): Linear(in_features=132, out_features=132, bias=True)
    (1): Tanh()
    (2): Dropout(p=0.5, inplace=False)
    (3): LayerNorm((132,), eps=1e-05, elementwise_affine=True)
    (4): Linear(in_features=132, out_features=66, bias=True)
    (5): Tanh()
    (6): Dropout(p=0.5, inplace=False)
    (7): LayerNorm((66,), eps=1e-05, elementwise_affine=True)
    (8): Linear(in_features=66, out_features=33, bias=True)
    (9): Tanh()
    (10): Dropout(p=0.5, inplace=False)
    (11): LayerNorm((33,), eps=1e-05, elementwise_affine=True)
    (12): Linear(in_features=33, out_features=16, bias=True)
    (13): Tanh()
    (14): Dropout(p=0.5, inplace=False)
    (15): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
    (16): Linear(in_features=16, out_features=8, bias=True)
    (17): Tanh()
    (18): Dropout(p=0.5, inplace=False)
    (19): LayerNorm((8,), eps=1e-05, elementwise_affine=True)
    (20): Linear(in_features=8, out_features=1, bias=True)
  )
)

Увы, при дальнейшей фильтрации молекул я брал высокооактивные молекулы, потому что не было сгенерировано ни одной молекулы с pIC50 > 6.0

In [15]:
from rdkit import Chem
from rdkit.Chem import QED
#from rdkit.Chem import SA_Score
from rdkit.Contrib.SA_Score import sascorer
#from rdkit.Chem import ToxAlertscd
from rdkit.Chem import FilterCatalog
from rdkit.Chem.FilterCatalog import FilterCatalogParams
from rdkit.Chem import Descriptors
import joblib

mordred_desc = mordred_desc.astype('float32')
tensor = torch.tensor(mordred_desc.values)
output = []

for iter in tensor:
    with torch.no_grad():
        output.append(float(model(iter)))

mols['pIC50_MLP'] = output

# QED (< 0.5)

qed_score = []

for mol in molecules:
    qed_score.append(QED.qed(mol))

mols['QED'] = qed_score

# SA Score

SA_score = []

for mol in molecules:
    SA_score.append(sascorer.calculateScore(mol))

mols['SA_score'] = SA_score

# Toxicophore

params = FilterCatalogParams()
params.AddCatalog(FilterCatalogParams.FilterCatalogs.BRENK)
catalog = FilterCatalog.FilterCatalog(params)

Toxicophore = []

for mol in molecules:   
    if catalog.HasMatch(mol):
        Toxicophore.append("Contains a Brenk alert!")
    else:
        Toxicophore.append("No Brenk alert")

mols['Toxicophore'] = Toxicophore

# Lipinski

Lipinski_list = []

for mol in molecules:

    mw = Descriptors.MolWt(mol)
    hba = Descriptors.NOCount(mol)
    hbd = Descriptors.NHOHCount(mol)
    logp = Descriptors.MolLogP(mol)

    if mw <= 500 and logp <= 5 and hbd <= 5 and hba <= 10:
        Lipinski_list.append("PASSES the Lipinski rule")
    else:
        Lipinski_list.append("VIOLATES the Lipinski rule")

mols['Lipinski'] = Lipinski_list

filtered_mols = mols[(mols['pIC50_MLP'] > 0.6) & (mols['QED'] >= 0.5) & (mols['SA_score'] >= 2) & (mols['SA_score'] <= 6) & (mols['Toxicophore'] == 'No Brenk alert') & (mols['Lipinski'] == "PASSES the Lipinski rule")]

filtered_mols.to_csv('Filtered_resultsMLP.csv', index = False)

In [28]:
# Использование RandomForestRegressor

model = joblib.load('RandomForestRegressor_model.joblib')

x = mordred_desc.astype('Float32')

y = model.predict(x)

mols['pIC50_RFR'] = y

# QED (< 0.5)

qed_score = []

for mol in molecules:
    qed_score.append(QED.qed(mol))

mols['QED'] = qed_score

# SA Score

SA_score = []

for mol in molecules:
    SA_score.append(sascorer.calculateScore(mol))

mols['SA_score'] = SA_score

# Toxicophore

params = FilterCatalogParams()
params.AddCatalog(FilterCatalogParams.FilterCatalogs.BRENK)
catalog = FilterCatalog.FilterCatalog(params)

Toxicophore = []

for mol in molecules:   
    if catalog.HasMatch(mol):
        Toxicophore.append("Contains a Brenk alert!")
    else:
        Toxicophore.append("No Brenk alert")

mols['Toxicophore'] = Toxicophore

# Lipinski

Lipinski_list = []

for mol in molecules:

    mw = Descriptors.MolWt(mol)
    hba = Descriptors.NOCount(mol)
    hbd = Descriptors.NHOHCount(mol)
    logp = Descriptors.MolLogP(mol)

    if mw <= 500 and logp <= 5 and hbd <= 5 and hba <= 10:
        Lipinski_list.append("PASSES the Lipinski rule")
    else:
        Lipinski_list.append("VIOLATES the Lipinski rule")

mols['Lipinski'] = Lipinski_list

filtered_mols = mols[(mols['pIC50_RFR'] > 0.1) & (mols['QED'] >= 0.5) & (mols['SA_score'] >= 2) & (mols['SA_score'] <= 6) & (mols['Toxicophore'] == 'No Brenk alert') & (mols['Lipinski'] == "PASSES the Lipinski rule")]

filtered_mols.to_csv('Filtered_resultsRandomForestRegressor.csv', index = False)

In [ ]:
# За кулисами обучил еще и CatBoostRegressor со значением r2 = 0.5

model = joblib.load('catboost_model.joblib')

x = mordred_desc.astype('Float32')

y = model.predict(x)

mols['pIC50_CBR'] = y

# QED (< 0.5)

qed_score = []

for mol in molecules:
    qed_score.append(QED.qed(mol))

mols['QED'] = qed_score

# SA Score

SA_score = []

for mol in molecules:
    SA_score.append(sascorer.calculateScore(mol))

mols['SA_score'] = SA_score

# Toxicophore

params = FilterCatalogParams()
params.AddCatalog(FilterCatalogParams.FilterCatalogs.BRENK)
catalog = FilterCatalog.FilterCatalog(params)

Toxicophore = []

for mol in molecules:   
    if catalog.HasMatch(mol):
        Toxicophore.append("Contains a Brenk alert!")
    else:
        Toxicophore.append("No Brenk alert")

mols['Toxicophore'] = Toxicophore

# Lipinski

Lipinski_list = []

for mol in molecules:

    mw = Descriptors.MolWt(mol)
    hba = Descriptors.NOCount(mol)
    hbd = Descriptors.NHOHCount(mol)
    logp = Descriptors.MolLogP(mol)

    if mw <= 500 and logp <= 5 and hbd <= 5 and hba <= 10:
        Lipinski_list.append("PASSES the Lipinski rule")
    else:
        Lipinski_list.append("VIOLATES the Lipinski rule")

mols['Lipinski'] = Lipinski_list

filtered_mols = mols[(mols['pIC50_CBR'] > 0.1) & (mols['QED'] >= 0.5) & (mols['SA_score'] >= 2) & (mols['SA_score'] <= 6) & (mols['Toxicophore'] == 'No Brenk alert') & (mols['Lipinski'] == "PASSES the Lipinski rule")]

filtered_mols.to_csv('Filtered_resultsCBR.csv', index = False)